# The Price of Hedging

1. Below we first generate 2 random return series we will use in the homwork.

In [ ]:
import numpy as np 
import pandas as pd 
def gen_strat_returns():
    np.random.seed(5)
    

    corr = [[1, -0.5],
            [-0.5, 1]]
    
    corr = np.array(corr)
    
    vols = np.diag(np.array([0.1, 0.1])) / np.sqrt(252)
    
    sigma = vols @ corr @ vols
    
    mu = np.array([0.1,0.1]) / 252
    
    dates = pd.date_range('20100101','20191231',freq='B')
    
    rets = np.random.multivariate_normal(mu, sigma, size = len(dates))
    rets = pd.DataFrame(rets,columns = ['X','HEDGE'], index = dates)
    

    rets = rets/rets.std()*0.1/np.sqrt(252)
    rets = rets-rets.mean()
    rets = rets+mu
    return rets

rets = gen_strat_returns()

2. Compute some basic stats on the return streams, including annualized returns, volatilities, sharpes and correlations

In [ ]:
stats = {}
stats['ret'] = rets.mean()*252
stats['vol'] = rets.std()*np.sqrt(252)
stats['sr'] = stats['ret'] / stats['vol']
stats = pd.DataFrame(stats)
stats

In [ ]:
rets.corr()

3. Combine the two returns streams optimally. What is the Sharpe ratio of the combo?

The sharpe ratio of the combo is enormous, owing to the large negative correlation (-0.5) between the two. 

In [ ]:
mu = rets.mean()
sigma = rets.cov()

def optimal_weights(sigma,mu):
    wgt = np.linalg.inv(sigma) @ mu 
    wgt = wgt / np.abs(wgt).sum()
    return wgt

wgt = optimal_weights(sigma,mu)
combo = (rets*wgt).sum(1)
combo.mean()/combo.std()*np.sqrt(252)

4. Now, compute what happens to the optimal weights and sharpes as we change the average annualized return on "HEDGE" in a range between -10% and 10% in increments of 1%. Roughly what return do you need on "HEDGE" to justify a zero weight? 

What is somewhat suprising here is that you need at least around a -5% return on "HEDGE" to justify a negative weight. Even between returns of -5% and 0%, "HEDGE" still desereves a positive weight. This is because of its extreme negative correlation with "X" and attractive hedging properties. Often, in the real world, hedges like this therefore cost money. The holy grail is finding a hedge that makes you money as well, which is difficult but doable.

In [ ]:

get_sharpe = lambda x: x.mean()/x.std()*np.sqrt(252)

wgts = {}
sharpes = {}
for hedge_ret in np.arange(-0.1,0.11,0.01):

    rets['HEDGE'] = rets['HEDGE'] - rets['HEDGE'].mean() + hedge_ret / 252
    mu = rets.mean() 
    
    wgt = pd.Series(optimal_weights(sigma,mu),mu.index)
    
    hedge_ret_str = '%.0f%%'%(hedge_ret*100)
    wgts[hedge_ret_str] = wgt 
    
    
    sharpes[hedge_ret_str] = get_sharpe((rets*wgt).sum(1))
    
wgts = pd.DataFrame(wgts).T
sharpes = pd.Series(sharpes)

In [ ]:
wgts

In [ ]:
wgts.plot(kind = 'bar')

In [ ]:
sharpes.plot()